# wrangling with the csv

In [2]:
import pandas as pd
import numpy as np
# import statsmodels.discrete.discrete_model as sm
import statsmodels as stt
import scipy.stats as sst
import os.path as osp

In [3]:
from statsmodels import api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import json
%matplotlib inline

### read the mapping file to find the correspondance between uberon numbers and labels

In [4]:
# find mapping file
# mapping_file = '../segstats_jsonld/segstats_jsonld/mapping_data/freesurfermap.json'
mapping_file = '../segstats_jsonld/segstats_jsonld/mapping_data/freesurfer-cdes.json'
assert osp.exists(mapping_file)
with open(mapping_file, "r") as read_file:
    roi_map = json.load(read_file)

In [9]:
print(len(list(roi_map.keys())))
print(list(roi_map.keys())[:10])
#print(list(roi_map['Anatomy'].keys())[:10])
#print(len(roi_map['Anatomy'].keys()))
print(roi_map[list(roi_map.keys())[10]])

3580
['count', "FS(structure='Cortex', hemi=None, measure='NumVert', unit='unitless')", "FS(structure='Cortex', hemi=None, measure='WhiteSurfArea', unit='mm^2')", "FS(structure='BrainSeg', hemi=None, measure='BrainSegVol', unit='mm^3')", "FS(structure='BrainSegNotVent', hemi=None, measure='BrainSegVolNotVent', unit='mm^3')", "FS(structure='BrainSegNotVentSurf', hemi=None, measure='BrainSegVolNotVentSurf', unit='mm^3')", "FS(structure='Cortex', hemi=None, measure='CortexVol', unit='mm^3')", "FS(structure='SupraTentorial', hemi=None, measure='SupraTentorialVol', unit='mm^3')", "FS(structure='SupraTentorialNotVent', hemi=None, measure='SupraTentorialVolNotVent', unit='mm^3')", "FS(structure='EstimatedTotalIntraCranialVol', hemi=None, measure='eTIV', unit='mm^3')"]
{'id': '000010', 'structure_id': 410, 'label': 'Right BA1_exvivo NumVert', 'description': 'Right BA1_exvivo Number of Vertices (unitless)', 'key_source': 'Table', 'measureOf': 'http://uri.interlex.org/base/ilx_0112559', 'datumTy

In [62]:
for k,v in roi_map.items():
    # print(k,"->",v)
    pass
v = {1:2}
print(type(v))
dict(v)

<class 'dict'>


{1: 2}

In [137]:
# ube is a dictionary that maps uberon to label of structure
ube = {}
ebu = {}
countok=0
has_no_isAbout = []
has_no_label = []

for (k,v) in roi_map.items():
    if k == 'count': pass

    # v is a dict that contains the CDE - check that we have a isAbout
    #if ('isAbout' not in v) or (v['isAbout'] == '') or (v['isAbout'] in ('None','none')):
    #    has_no_isAbout.append(k)
    #    break
        
    elif 'isAbout' in v:
        countok += 1
        if 'label' in v:
            if v['label'] != '' and v['label'] not in ('None','none'):
                #ube['<' + v['isAbout'] + '>'] = v['label']
                #ebu[v['label']] = '<' + v['isAbout'] + '>'
                ebu[v['label']] = v['isAbout']
                if v['isAbout'] not in ube.keys():
                    no_right_or_left = v['label']
                    no_right_or_left = no_right_or_left.replace('Right-','')
                    no_right_or_left = no_right_or_left.replace('Right ','')
                    no_right_or_left = no_right_or_left.replace('Left-','')
                    no_right_or_left = no_right_or_left.replace('Left ','')
                    no_right_or_left = no_right_or_left.replace(' NVoxels','')
                    no_right_or_left = no_right_or_left.replace(' (mm^3)','')
                    ube[v['isAbout']] = no_right_or_left
            else:
                has_no_label.append(k)

print(has_no_isAbout)
#print(ube)
#print(ebu)
print(countok , len(ebu))

[]
262 262


In [138]:
ube

{'http://purl.obolibrary.org/obo/UBERON_0000956': 'Number of Vertices',
 'http://purl.obolibrary.org/obo/UBERON_0000955': 'Brain Segmentation Volume',
 'http://purl.obolibrary.org/obo/UBERON_0001869': 'Supratentorial volume',
 'http://purl.obolibrary.org/obo/UBERON_0013411': 'Estimated Total Intracranial Volume',
 'http://purl.obolibrary.org/obo/UBERON_0002437': 'hemisphere cerebral white matter volume',
 'http://purl.obolibrary.org/obo/UBERON_0005401': 'Total gray matter volume',
 'http://purl.obolibrary.org/obo/UBERON_0002285': 'Lateral-Ventricle',
 'http://purl.obolibrary.org/obo/UBERON_0006091': 'Inf-Lat-Vent',
 'http://purl.obolibrary.org/obo/UBERON_0002317': 'Cerebellum-White-Matter',
 'http://purl.obolibrary.org/obo/UBERON_0002129': 'Cerebellum-Cortex',
 'http://purl.obolibrary.org/obo/UBERON_0001897': 'Thalamus-Proper',
 'http://purl.obolibrary.org/obo/UBERON_0001873': 'Caudate',
 'http://purl.obolibrary.org/obo/UBERON_0001874': 'Putamen',
 'http://purl.obolibrary.org/obo/UBERO

In [116]:
#
"""
ube = {}
has_no_isAbout = []
for (k,v) in roi_map['Anatomy'].items():
    if v['isAbout'] != '' or v['isAbout'] in ('None','none'):
        if v['label'] != '' and v['label'] not in ('None','none'):
            ube['<' + v['isAbout'] + '>'] = v['label']
        else:
            if '<' + v['isAbout'] + '>' not in ube.keys():
                if v['hasLaterality'] in ('Right','Left'):
                    ube['<' + v['isAbout'] + '>'] = 'strip-Right-Left-of-'+k
                else:
                    ube['<' + v['isAbout'] + '>'] = k
    else:
        #print(k, 'has no isAbout')
        has_no_isAbout.append(k)
print(has_no_isAbout)
""";

In [117]:
list(zip(list(ube)[:5],list(ube.values())[:5]))

[('http://purl.obolibrary.org/obo/UBERON_0000956', 'Number of Vertices'),
 ('http://purl.obolibrary.org/obo/UBERON_0000955',
  'Brain Segmentation Volume'),
 ('http://purl.obolibrary.org/obo/UBERON_0001869', 'Supratentorial volume'),
 ('http://purl.obolibrary.org/obo/UBERON_0013411',
  'Estimated Total Intracranial Volume'),
 ('http://purl.obolibrary.org/obo/UBERON_0002437',
  'Left hemisphere cerebral white matter volume')]

### Read CSV file 

In [118]:
print(osp.realpath(osp.curdir))
relative_path_filename = './data/test.csv'
assert osp.exists(relative_path_filename)

/home/jb/code/repronim/simple2/simple2_analysis


In [119]:
hie = pd.read_csv(relative_path_filename, na_values='nd')
original_col_names = list(hie)
print(original_col_names)
assert len(original_col_names) == len(set(original_col_names))

['Unnamed: 0', 'ID', 'Age', 'dx', 'Gender', 'FIQ', 'PIQ', 'VIQ', 'tool', 'softwareLabel', 'federatedLabel', 'laterality', 'volume']


In [120]:
structures = set(hie.federatedLabel)
print(structures)

{'http://purl.obolibrary.org/obo/UBERON_0001874', 'http://purl.obolibrary.org/obo/UBERON_0001954', 'http://purl.obolibrary.org/obo/UBERON_0001876', 'http://purl.obolibrary.org/obo/UBERON_0000956', 'http://purl.obolibrary.org/obo/UBERON_0001882', 'http://purl.obolibrary.org/obo/UBERON_0002437', 'http://purl.obolibrary.org/obo/UBERON_0001873', 'http://purl.obolibrary.org/obo/UBERON_0001981', 'http://purl.obolibrary.org/obo/UBERON_0006091', 'http://purl.obolibrary.org/obo/UBERON_0001886', 'http://purl.obolibrary.org/obo/UBERON_0002285', 'http://purl.obolibrary.org/obo/UBERON_0002317', 'http://purl.obolibrary.org/obo/UBERON_0006514', 'http://purl.obolibrary.org/obo/UBERON_0002129', 'http://purl.obolibrary.org/obo/UBERON_0001897'}


In [128]:
ube2h = {}

for idx, s in enumerate(structures):
    ube2h[s] = ube[s]
    print(idx, s, ube2h[s])

len(ube), len(ube2h)

0 http://purl.obolibrary.org/obo/UBERON_0001874 Putamen
1 http://purl.obolibrary.org/obo/UBERON_0001954 Hippocampus
2 http://purl.obolibrary.org/obo/UBERON_0001876 Amygdala
3 http://purl.obolibrary.org/obo/UBERON_0000956 Number of Vertices
4 http://purl.obolibrary.org/obo/UBERON_0001882 Accumbens-area
5 http://purl.obolibrary.org/obo/UBERON_0002437 Left hemisphere cerebral white matter volume
6 http://purl.obolibrary.org/obo/UBERON_0001873 Caudate
7 http://purl.obolibrary.org/obo/UBERON_0001981 vessel
8 http://purl.obolibrary.org/obo/UBERON_0006091 Inf-Lat-Vent
9 http://purl.obolibrary.org/obo/UBERON_0001886 choroid-plexus
10 http://purl.obolibrary.org/obo/UBERON_0002285 Lateral-Ventricle
11 http://purl.obolibrary.org/obo/UBERON_0002317 Cerebellum-White-Matter
12 http://purl.obolibrary.org/obo/UBERON_0006514 Pallidum
13 http://purl.obolibrary.org/obo/UBERON_0002129 Cerebellum-Cortex
14 http://purl.obolibrary.org/obo/UBERON_0001897 Thalamus-Proper


(28, 15)

In [ ]:
"""
looking at the class name from the purl urls I got :

u2human[list(u2human.keys())[0]] = 'cavum septum pellucidum'
u2human[list(u2human.keys())[1]] = 'cerebellum'
u2human[list(u2human.keys())[2]] = 'cerebellar cortex'
u2human[list(u2human.keys())[3]] = 'perivascular space'
u2human[list(u2human.keys())[4]] = "Ammon's horn"
u2human[list(u2human.keys())[5]] = "inferior horn of the lateral ventricle"
u2human[list(u2human.keys())[6]] = "brainstem"
u2human[list(u2human.keys())[7]] = "amygdala"
u2human[list(u2human.keys())[8]] = "optic chiasma"
u2human[list(u2human.keys())[9]] = "dorsal plus ventral thalamus"
u2human[list(u2human.keys())[10]] = "cerebral hemisphere"
u2human[list(u2human.keys())[11]] = "caudate nucleus"
u2human[list(u2human.keys())[12]] = "telencephalic ventricle"
u2human[list(u2human.keys())[13]] = "putamen"
u2human[list(u2human.keys())[14]] = "third ventricle"
u2human[list(u2human.keys())[15]] = "choroid plexus"
u2human[list(u2human.keys())[16]] = "fourth ventricle"
u2human[list(u2human.keys())[17]] = "pallidum"
u2human[list(u2human.keys())[18]] = "nucleus accumbens"
""";

In [ ]:
set1 = {'<http://purl.obolibrary.org/obo/UBERON_0000959>', 
'<http://purl.obolibrary.org/obo/UBERON_0002422>', 
'<http://purl.obolibrary.org/obo/UBERON_0009857>', 
'<http://purl.obolibrary.org/obo/UBERON_0001869>', 
'<http://purl.obolibrary.org/obo/UBERON_0002129>', 
'<http://purl.obolibrary.org/obo/UBERON_0001882>', 
'<http://purl.obolibrary.org/obo/UBERON_0001886>', 
'<http://purl.obolibrary.org/obo/UBERON_0006091>', 
'<http://purl.obolibrary.org/obo/UBERON_0002286>', 
'<http://purl.obolibrary.org/obo/UBERON_0002298>', 
'<http://purl.obolibrary.org/obo/UBERON_0001897>', 
'<http://purl.obolibrary.org/obo/UBERON_0001954>', 
'<http://purl.obolibrary.org/obo/UBERON_0001873>', 
'<http://purl.obolibrary.org/obo/UBERON_0014930>', 
'<http://purl.obolibrary.org/obo/UBERON_0001874>', 
'<http://purl.obolibrary.org/obo/UBERON_0001876>', 
'<http://purl.obolibrary.org/obo/UBERON_0006514>', 
'<http://purl.obolibrary.org/obo/UBERON_0002037>', 
'<http://purl.obolibrary.org/obo/UBERON_0002285>'};
set2 = set(ube2h.keys())
set1.difference(set2)

In [123]:
h2ube = {v: k for k, v in ube2h.items()}
#print(h2uberon)

In [125]:
# print(set(hie['tool']))
tooldic = {'surfer':'https://surfer.nmr.mgh.harvard.edu/', 
           'fsl':'http://purl.org/nidash/fsl#'}
print(tooldic)

{'surfer': 'https://surfer.nmr.mgh.harvard.edu/', 'fsl': 'http://purl.org/nidash/fsl#'}


In [126]:
hie.head(3)

,Unnamed: 0,ID,Age,dx,Gender,FIQ,PIQ,VIQ,tool,softwareLabel,federatedLabel,laterality,volume
0,0,50201,13.77,2,Female,98.0,90.0,106.0,https://surfer.nmr.mgh.harvard.edu/,Left-Lateral-Ventricle Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0002285,Left,10337.7
1,1,50201,13.77,2,Female,98.0,90.0,106.0,https://surfer.nmr.mgh.harvard.edu/,Left-Amygdala Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001876,Left,1457.7
2,2,50201,13.77,2,Female,98.0,90.0,106.0,https://surfer.nmr.mgh.harvard.edu/,Right-Thalamus-Proper Volume_mm3 (mm^3),http://purl.obolibrary.org/obo/UBERON_0001897,Right,7611.3


## hypotheses

Hypotheses

PIET-1: Total Brain Volume will positively correlate with IQ (in both sexes across the complete age range).

MAC-1: Left striatum volume (caudate + putamen) will positively correlate with IQ in the total (male + female) child (age < 20) group.

MAC-2: Left striatum volume (caudate + putamen) will positively correlate with IQ in the male children group.

MAC-3: Left striatum volume (caudate + putamen) will not correlate with IQ in the female children group.

MAC-3: Left striatum volume (caudate + putamen) will not correlate with IQ in the female children group.

GANJ-1: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ.

GANJ-2: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ in the young (age < 12) group.

GANJ-3: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will not significantly correlate with IQ in the adolescent (age > 12) group.

GANJ-4:. Total Corpus Callosum midsagittal area, after correcting for total brain volume, will negatively correlate with IQ in the male (age < 12) group.

GANJ-5: Total Corpus Callosum midsagittal area, after correcting for total brain volume, will not significantly correlate with IQ in the female (age < 12) group.


## Analyses

### sandbox

In [129]:
u_caudate = h2ube['Caudate']
print(u_caudate)

http://purl.obolibrary.org/obo/UBERON_0001873


In [132]:
# df.loc[(df['column_name'] >= A) & (df['column_name'] <= B)]

# tmp = hie[hie.structure==u_caudate][hie.Gender=='1']
# tmp = hie.loc[(hie['Gender']=='1')] # & (hie['Gender']=='Male')]

# Male or '1' for caudate
tmp = hie.loc[((hie['Gender']=='1') | (hie['Gender']=='Male')) 
                               & (hie['federatedLabel'] == u_caudate ) 
                               & (hie['tool']==tooldic['surfer'])]


In [136]:
print(set(hie['tool']))
#print(tooldic['fsl'])
#print(len(hie),len(tmp))
#print(set(hie['hand']))
print(set(hie['Gender']))
#print(set(hie['dx']))
print(set(hie['laterality']))


{'https://surfer.nmr.mgh.harvard.edu/', 'http://purl.org/nidash/fsl#'}
{'Male', 'Female'}
{'Left', 'Right'}


In [ ]:
h2ube

### PIET-1: Total Brain Volume will positively correlate with IQ (in both sexes across the complete age range).


In [130]:
roi = 'cerebellar cortex'
# roi = 'cerebellar cortex'
tool = 'surfer'
#tool = 'fsl'

tmp = hie[['study','ID','Gender','structure','tool','value','piq','fiq']]
# tmp = tmp.loc[((hie['Gender']=='1') | (hie['Gender']=='Male')) 

# hand_cond = ((hie['hand'] == 'L')|(hie['hand'] == 'Left')|(hie['latera'] == 'Left'))
age_cond = (hie['Age'] <= 20)
roi_cond = (hie['structure'] == h2ube[roi])
tool_cond = (hie['tool']==tooldic[tool])

condition = roi_cond & tool_cond # & age_cond 

tmp = tmp.loc[condition].drop_duplicates()

print(len(hie),len(tmp))

KeyError: "['structure', 'fiq', 'value', 'study', 'piq'] not in index"

In [ ]:
list(tmp)

In [ ]:
print(" Structure = ", roi)
assert ube2h[tmp.iloc[0]['structure']] == roi

iq = 'fiq'

md = smf.ols(iq + " ~ Q('value') + Gender + study ", data=tmp) # + study 
mdf = md.fit()
print(mdf.summary())


### conclusion : based on the wrong structure ...
1. no significance for piq for what seems to be the closest region:
Q('value')                      -0.0410      0.032     -1.286      0.199    
1. negatively correlated for fiq for what seems to be the closest region:
Q('value')                      -0.1232      0.032     -3.881      0.000      

### MAC-1: Left striatum volume (caudate + putamen) will positively correlate with IQ in the total (male + female) child (age < 20) group.


In [ ]:
roi1 = 'caudate nucleus'
roi2 = 'putamen'
# roi = 'cerebellar cortex'
tool = 'surfer'
# tool = 'fsl'

# pick the interesting variables
#--------------------------------#
tmp = hie[['study','ID','Gender','structure','tool','value', 'fiq', 'piq', 'latera']]
# condition1 = ((hie['Gender']=='1') | (hie['Gender']=='Male')) & 

# list of conditions
#---------------------#
hand_cond = ((hie['hand'] == 'L') | (hie['hand'] == 'Left')) # | (hie['latera'] == 'Left'))
age_cond = (hie['Age'] <= 20)
roi1_cond = (hie['structure'] == h2ube[roi1])
roi2_cond = (hie['structure'] == h2ube[roi2])
tool_cond = (hie['tool']==tooldic[tool])

condition1 = roi1_cond & tool_cond & age_cond & hand_cond  
condition2 = roi2_cond & tool_cond & age_cond & hand_cond

tmp1 = tmp.loc[condition1].drop_duplicates()
tmp2 = tmp.loc[condition2].drop_duplicates()

#tmp1 = tmp1.set_index('ID')
#tmp2 = tmp2.set_index('ID')

print(len(hie))
print(roi1, ': tmp1', len(tmp1), len(tmp1.drop_duplicates()))
print(roi2, ': tmp2', len(tmp2), len(tmp2.drop_duplicates()))

In [ ]:
tmp2

In [ ]:
def split_merge_df(df, indx='ID', spliton='latera', cols='value', droplist=[]):
    """
    split the df, create a new one merged
    think of lev1 = Right, lev2 = Left
    """
    levels_ = list(set(df[spliton]))
    # print(levels_)
    levels = [v for v in levels_ if v != 'None']
    assert len(levels)==2
    lev1, lev2 = levels[0], levels[1]
    dflev1 = df[df[spliton]==lev1]
    dflev2 = df[df[spliton]==lev2]
    
    # no duplicates in the indx
    assert len(set(dflev1[indx])) == len(dflev1[indx])
    assert len(set(dflev2[indx])) == len(dflev2[indx])
    # same indx on all levels
    assert len(dflev1[indx]) == len(dflev2[indx])
    
    merged_inner = pd.merge(left=dflev1, right=dflev2, left_on=indx, right_on=indx)
    merged_inner.rename(columns={cols+'_x': cols+'_'+lev1, cols+'_y': cols+'_'+lev2}, inplace=True)
    
    if droplist != []:
        for colname in droplist:
            colname_y = colname + '_y'
            colname_x = colname + '_x'
            merged_inner.drop(colname_y, axis=1, inplace=True)
            merged_inner.rename(columns={colname_x: colname}, inplace=True)
    
    return merged_inner

In [ ]:
droplist = list(tmp)
for colname in ['ID', 'value', 'latera']:
    droplist.remove(colname)
print(droplist)
tmp_1 = split_merge_df(tmp1, indx='ID', spliton='latera', cols='value',droplist=droplist)
tmp_2 = split_merge_df(tmp2, indx='ID', spliton='latera', cols='value',droplist=droplist)

In [ ]:
len(tmp_1)

In [ ]:
# a function that takes the laterality right and create a new column
df = tmp1.copy()

list(df)
dfleft = df[df['latera']=='Left'].copy()
dfright = df[df['latera']=='Right']

# check IDs are unique and the same for left and right
assert len(set(dfleft['ID'])) == len(dfleft['ID'])
assert len(set(dfright['ID'])) == len(dfright['ID'])
assert set(dfright['ID']) == set(dfleft['ID'])

# create a new column on dfleft
#dfleft.set_index('ID');
#dfright.set_index('ID');
merged_inner = pd.merge(left=dfleft, right=dfright, left_on='ID', right_on='ID')

# -- try to drop duplicates 
# to do: https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/
# merged_inner = merged_inner.loc[:,~merged_inner.columns.duplicated()]

In [ ]:
set(df['latera'])

In [ ]:
merged_inner.head()

In [ ]:
import numbers
import decimal
hand_nb = [v for v in hie['hand'] if isinstance(v, numbers.Number)]
len(hand_nb)

In [ ]:
ax = plt.hist(tmp1['value'])

In [ ]:
ax = plt.hist(tmp2['value'])

In [ ]:
# tmp2.head(2)

In [ ]:
tmp1.head(5)

In [ ]:
# hie.loc[(hie['ID']==21002) & (hie['structure']==h2uberon[roi1]) & (hie['tool']==tooldic['surfer'])]

#### try to add values of the same index

In [ ]:
datadf = tmp1[tmp1['latera'] == 'Left' ]
datadf.set_index('ID', inplace=True)
len(datadf)
datadf.head(3)
datadf_right = tmp1[tmp1['latera'] == 'Right' ]
datadf['value_right'] = datadf_right['value']

In [ ]:
tmp_1 = tmp1.groupby(['ID']).mean().dropna()
tmp_2 = tmp2.groupby(['ID']).mean().dropna()
print(len(tmp_2), len(tmp_1))
assert np.all(tmp_1.index == tmp_2.index)

In [ ]:
tmp_2.head()

In [ ]:
""" print(type(tmp_2))
print(list(tmp_2))
print(tmp_2['value'][:5])
print(tmp_1['value'][:5]) 

list(tmp_2)
len(set(tmp_2.index))


lists_index = [set(tmp1['ID']),  set(tmp2['ID'])]
len( set.intersection(*lists_index))
""";

In [ ]:
datadf1 = tmp1.set_index('ID')
datadf2 = tmp2.set_index('ID')
len(datadf1), len(datadf2)

In [ ]:
datadf = pd.merge(left=tmp1,right=tmp2, left_on='ID', right_on='ID')

In [ ]:
datadf.head()

In [ ]:

datadf['cau_put'] = tmp_2.loc[:,'value'].add(tmp_1.loc[:,'value'])
datadf[iq] = tmp_2[iq]
datadf['study'] = tmp_2['study']
datadf['Gender'] = tmp_2['Gender']

print(list(datadf))

In [ ]:
#datadf = datadf[datadf['cau_put'] < 15000]
len(datadf)
ax = plt.hist(datadf['cau_put'])

In [ ]:
iq = 'piq'

md = smf.ols(iq + " ~ Q('cau_put') + study + Gender", data=datadf)
mdf = md.fit()
print(mdf.summary())


### Conclusion

1. does not seem to replicate with fiq (only 58 values ?)
2. does not seem to replicate with piq


### MAC-2: Left striatum volume (caudate + putamen) will positively correlate with IQ in the male children group.


In [ ]:
#merged_inner = pd.merge(left=tmp_1, right=tmp_2) #, left_on='ID', right_on='ID') #how='join', 

In [ ]:
roi1 = 'caudate nucleus'
roi2 = 'putamen'
# roi = 'cerebellar cortex'
tool = 'surfer'
# tool = 'fsl'

# tmp = hie[['ID','Gender','structure','tool','value','fiq']]
# condition1 = ((hie['Gender']=='1') | (hie['Gender']=='Male')) & 

condition1 = (hie['structure'] == h2uberon[roi1]) & (hie['tool']==tooldic[tool]) \
             & (hie['Age'] < 20) \
             & ((hie['Gender']=='1') | (hie['Gender']=='Male'))
condition2 = (hie['structure'] == h2uberon[roi2]) & (hie['tool']==tooldic[tool]) \
             & (hie['Age'] < 20) \
             & ((hie['Gender']=='1') | (hie['Gender']=='Male'))

    
tmp1 = tmp.loc[condition1].drop_duplicates()
tmp2 = tmp.loc[condition2].drop_duplicates()
#tmp1 = tmp1.set_index('ID')
#tmp2 = tmp2.set_index('ID')


In [ ]:
print(len(hie),len(tmp1),len(tmp2))
print(len(tmp1.drop_duplicates()),len(tmp2.drop_duplicates()))

In [ ]:
tmp_1 = tmp1.groupby(['ID']).mean().dropna()
tmp_2 = tmp2.groupby(['ID']).mean().dropna()
print(len(tmp_2), len(tmp_1))
assert np.all(tmp_1.index == tmp_2.index)

In [ ]:
datadf = tmp_2
datadf['cau_put'] = tmp_2.loc[:,'value'].add(tmp_1.loc[:,'value'])
datadf['fiq'] = tmp_2['fiq']
print(list(datadf))

In [ ]:
print(len(datadf))
# ax = plt.hist(datadf['cau_put'])

In [ ]:

md = smf.ols("fiq ~ Q('cau_put')", data=datadf)
mdf = md.fit()
print(mdf.summary())


### 

In [ ]:
#tmp = hie.loc[ (hie['structure']==h2uberon[roi]) & (hie['tool']==tooldic['surfer'])]
#tmp.head(3)

In [ ]:
#tmp_ = tmp1.groupby(['ID']).min()
#tmp_.head(3)

In [ ]:
# horizontal_stack = pd.concat([tmp1, tmp2], axis=1)
merged_inner = pd.merge(left=tmp1, right=tmp2, left_on='ID', right_on='ID') #how='join', 

In [ ]:
len(merged_inner.drop_duplicates())

In [ ]:
merged_inner.head(4)

In [ ]:
md = smf.ols("FIQ ~ Q('striatal volume')", data=data2)
mdf = md.fit()
print(mdf.summary())
print(pearsonr(data2['FIQ'], data2['striatal volume']))

In [ ]:
print(tmp.loc[(tmp['ID']==3559087)])

In [ ]:
hie.iloc[1578]

In [ ]:
hie.iloc[1577]

In [ ]:
uberon = "http://purl.obolibrary.org/obo/UBERON_0001897"

cmd = 'wget -O - ' + uberon\
                   + '  | grep -A 1 ' +  "'" + 'Class rdf:about='\
                   + '"' + uberon + '"' +  "'"

In [ ]:
print(cmd.split())

In [ ]:
import subprocess
MyOut = subprocess.Popen(cmd.split(), 
            stdout=subprocess.PIPE, 
           stderr=subprocess.STDOUT, shell=True)
stdout,stderr = MyOut.communicate()
print(stdout)
#print(stderr)

In [ ]:
process = subprocess.run(cmd.split())

In [ ]:
list(hie)

In [ ]:
uberon= '<http://purl.obolibrary.org/obo/UBERON_0001897>'


In [ ]:
len(tmp)

In [ ]:
# Load in the dataset, appending 'nd' to the list of possible na_values
hie = pd.read_csv(relative_path_filename, na_values='nd')
original_col_names = list(hie)
print(original_col_names[:10])

# column names are unique
assert len(original_col_names) == len(set(original_col_names))